# ETL Pipeline: Top 10 Largest Banks
**Project Description:**
This notebook implements an automated ETL (Extract, Transform, Load) pipeline to compile a list of the top 10 largest banks in the world by market capitalization. The data is scraped from Wikipedia, transformed using current exchange rates for GBP, EUR, and INR, and loaded into both a CSV file and a SQLite database.

**Author:** Malik Tahayneh

**Date:** December 2025

In [ ]:
# Install required libraries (Run this once if needed)
!pip install requests bs4 pandas numpy lxml

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import sqlite3
import numpy as np
from datetime import datetime

remote_url = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-PY0221EN-Coursera/labs/v2/exchange_rate.csv'
local_csv_name = 'exchange_rate.csv'

try:
    response = requests.get(remote_url)
    if response.status_code == 200:
        with open(local_csv_name, 'wb') as f:
            f.write(response.content)
        print(f"Success: {local_csv_name} downloaded.")
    else:
        print("Error: Failed to download CSV.")
except Exception as e:
    print(f"Error: {e}")


url = 'https://web.archive.org/web/20230908091635/https://en.wikipedia.org/wiki/List_of_largest_banks'
exchange_rate_path = f'./{local_csv_name}'  # Point to the downloaded file
table_attribs_extraction = ['Name', 'MC_USD_Billion']
output_csv_path = './Largest_banks_data.csv'
db_name = 'Banks.db'
table_name = 'Largest_banks'
log_file = 'code_log.txt'

In [ ]:
def log_progress(message):
    ''' Logs the execution stage to a file with a timestamp. '''
    timestamp_format = '%Y-%h-%d-%H:%M:%S' 
    now = datetime.now() 
    timestamp = now.strftime(timestamp_format) 
    
    with open(log_file,"a") as f: 
        f.write(timestamp + ' : ' + message + '\n')
        
    print(f"Logged: {message}")

In [ ]:
def extract(url, table_attribs):
    ''' Scrapes the website and returns a dataframe with Bank Name and Market Cap (USD). '''
    
    page = requests.get(url).text
    data = BeautifulSoup(page, 'html.parser')
    
    df_list = []
    
    # Locate the first table under 'By market capitalization'
    tables = data.find_all('tbody')
    rows = tables[0].find_all('tr') 
    
    for row in rows:
        col = row.find_all('td')
        if len(col) != 0:
            if col[1].find('a') is not None:
                name = col[1].find('a').contents[0]
                market_cap = col[2].contents[0]
                market_cap = float(market_cap.strip())
                
                df_list.append({table_attribs[0]: name, table_attribs[1]: market_cap})

    df = pd.DataFrame(df_list, columns=table_attribs)
    return df

In [ ]:
def transform(df, csv_path):
    ''' Adds columns for GBP, EUR, and INR based on exchange rates. '''
    
    # Read exchange rate CSV
    exchange_rate = pd.read_csv(csv_path)
    
    # Convert to dictionary: {'Currency': Rate}
    exchange_rate_dict = exchange_rate.set_index('Currency').to_dict()['Rate']
    
    # Add new columns
    df['MC_GBP_Billion'] = [np.round(x * exchange_rate_dict['GBP'], 2) for x in df['MC_USD_Billion']]
    df['MC_EUR_Billion'] = [np.round(x * exchange_rate_dict['EUR'], 2) for x in df['MC_USD_Billion']]
    df['MC_INR_Billion'] = [np.round(x * exchange_rate_dict['INR'], 2) for x in df['MC_USD_Billion']]
    
    return df

In [ ]:
def load_to_csv(df, output_path):
    ''' Saves the dataframe to a CSV file. '''
    df.to_csv(output_path, index=False)

def load_to_db(df, sql_connection, table_name):
    ''' Saves the dataframe to a SQL database table. '''
    df.to_sql(table_name, sql_connection, if_exists='replace', index=False)

def run_query(query_statement, sql_connection):
    ''' Runs a SQL query and prints the output. '''
    print(f"\nQuery: {query_statement}")
    query_output = pd.read_sql(query_statement, sql_connection)
    print(query_output)

In [ ]:
# --- Execution Block ---

# 1. Start Process
log_progress('Preliminaries complete. Initiating ETL process')

# 2. Extract
df = extract(url, table_attribs_extraction)
log_progress('Data extraction complete. Initiating Transformation process')
print("\nExtracted Data (First 5 rows):")
display(df.head()) 

# 3. Transform
df = transform(df, exchange_rate_path)
log_progress('Data transformation complete. Initiating Loading process')
print("\nTransformed Data (First 5 rows):")
display(df.head())

# 4. Load to CSV
load_to_csv(df, output_csv_path)
log_progress('Data saved to CSV file')

# 5. Load to Database
sql_connection = sqlite3.connect(db_name)
log_progress('SQL Connection initiated.')

load_to_db(df, sql_connection, table_name)
log_progress('Data loaded to Database as table. Running the query')

# 6. Run Queries
query_1 = f"SELECT * from {table_name}"
run_query(query_1, sql_connection)

query_2 = f"SELECT AVG(MC_GBP_Billion) FROM {table_name}"
run_query(query_2, sql_connection)

query_3 = f"SELECT Name from {table_name} LIMIT 5"
run_query(query_3, sql_connection)

log_progress('Process Complete.')

# 7. Close Connection
sql_connection.close()